# Eclipse Setup

---
## Installing Eclipse
You can pull Eclipse using "apt-get install", but I prefer to get it directly from the Eclipse site. That way I get the latest, and I manage the update from Eclipse itself. 
- Go to the Eclipse packages page (https://www.eclipse.org/downloads/packages/)
- Select "Eclipse IDE for C/C++ Developers"and download the Linux version
- Once downloaded, double click the .tgz package. 
- My installation (I would recommend something similar)
  - Extract the "eclipse/" directory into my "home", i.e. /home/"user" (/home/martin/eclipse in my case)
  - Create a "launcher", i.e. a desktop icon like this:
    - Right click anywhere on the Linux desktop
    - Select "create launcher"
    - Name: Eclipse
    - Description: leave empty
    - Command: /home/martin/eclipse

---
## Workspaces
When you run Eclipse the first time, it want a workspace. A workspace is where your projects are stored. You can create one in your "home", but I use a workspace at the base directory. It can help with shorter path lengths. You will need to change ownership to you, i.e. "user". To do this follow the setup below, then set the Eclipse workspace to it. Note, if you use the same workspace path as me, and the same compiler directory, then any example projects will work in your environment without modification.
```
cd /
mkdir workspace
chown -R martin /workspace
```

---
## EGit
This is the Eclipse foundation standard pluging for **git**. Once Eclipse is up an running:
- Select "Help" from the menu bar
- Select "Eclipse Marketplace"
- Search for "EGit"
- Select "Install" from the EGit panel, and use the defaults
- Restart Eclipse

----
## Creating BBB3 project
This assumes you are using the workspace directory **/workspace** and have managed ownership with **chown**. There are two steps to this:
- clone from github
- import to Eclipse

#### Clone BBB3
```
cd /workspace
git clone https://github.com/mg4news/BBB3.git
```

#### Importing the project
From Eclipse:
- Select **File** from the menu bar
- Select **Import** from the drop down
- Choose C/C++, Existing code as a makefile project, select Next..
- In the **Import Existing Code** dialog:
  - Ignore **Project Name**
  - Select **Browse** beside **Existing Code Location**
  - Navigate to the code, should be /workspace/BBB3, select it. This will also populate the **Project Name** with BBB3
  - In **Languages** select both C and C++
  - In **Toolchain for Indexer settings** select the **Linux GCC** toolchain. This is not for compiling, just to resolve symbols and includes in the editor. 
  - Select **Finish**

You should now have the BBB3 project in the Eclipse Project Explorer. It will already be annotated as a git project. You will be able to:
- make local commits
- pull updates from the github repository

You **will not be able** to push updates back to github.

---
## Creating makefile targets
If you are using all of the paths described in these notebooks then you will be able to build the applications from the command line, as follows:
- In the terminal, cd to /workspace/apps/[appname]
- "make clean" will delete all the build products (.o, executables, etc)
- "make all" will build the executable
- "make strip" will build the executable and strip out all excess symbols, i.e. small executable for production

#### Eclipse targets
- In Eclipse select the **Build Targets** tab from the right hand pane
- For now make sure the **Hide Empty Folders** icon is **unselected** (the right most icon, a folder with a strikethrough)
- Navigate to an app directory, i.e. BBB3->apps->"appname"
- Right click the app directory and select **New**
  - Target name: Descriptive name, i.e. Clean, Build, Build Stripped, etc
  - Uncheck the **Same as the target name** checkbox
  - Leave the **Build Target** box empty
  - Uncheck the **Use builder settings** checkbox
  - In the **Build Command** box enter one of:
    - make clean
    - make --quiet all
    - make --quiet strip
  - Check the **Stop on first build error** checkbox
  - Uncheck the **Run all project builders** checkbox
- Do this for all three target types

Once you have finished, test by double clicking each of the targets. You can select the **Hide Empty Folders** icon, then you only see a folders with targets

---
## Remote Execution
For this example, assume we are building a target for BBB3/apps/gpio/gpio
- Build the app. This will cause some of the options to default correctly as you add the run target
- From the Eclipse menu, select the down arrow beside the "Run" icon (green circle with white arrow in it)
- Select **Run Configurations**
- Select **C/C++ Remote Application**
- Click the **New Configuration** icon (top left)
  - Name: "BBB3 gpio test". This is a descriptive name, you could use something different
  - Project: BBB3 (if you are using this project structure and name)
  - C/C++ application: apps/gpio/gpio (should already be populated, if not navigate to the **executable**
  - Build (if required) before launching: Disable by selecting **Disable auto build**
  - Connection: Create new (select **New...*). Note, you will use this with all run/debug targets
    - Connection type: choose **SSH**
    - Connection Name: I used **BBB3**
    - Host: IP address of BBB3 
    - User: debian
    - Select **Password based authentication**
    - Password: temppwd
    - **Finish**. This will put you back in the main remote application dialog again
  - Remote Absolute File Path for C/C++ Application: Select **Browse.**. This will connect to the BBB3. Browse to "tests", double click to select. click **OK**
    - this should result in "/home/debian/gpio" being the path. If it is not then you can simply manually edit the path
  - Commands to execute before applicatino: Leave empty
  - Skip download to target path: Leave unchecked
  - **Apply**

If you run this target it will push the executable to the BBB3, and run it. Any output will appear in the Eclipse console. 

---
## Remote Debug
For this example, assume we are building a target for BBB3/apps/gpio/gpio, AND that you are using the same workspace directory and toolchain path. 
- Build the app. This will cause some of the options to default correctly as you add the run target. **Do not use the stripped build**, this will remove debug symbols
- From the Eclipse menu, select the down arrow beside the "Debug" icon (green bug)
- Select **Debug Configurations**. You should see the target you just created for remote execution. The only thing needed is to set up the remote GDB stuff. You already have **gdbserver** installed on the BBB3 as part of the pre-requisites.
- Select the **BBB3 gpio test** remote application
- Select the **Debugger** tab
- Change the **GDB Debugger** to "/workspace/gcc-bbb3/bin/arm-linux-gnueabihf-gdb"
- **Apply**

If you run this target it will launch the remote application, change the Eclipse perspective to debug, and will pause the remote debug on the entry to main(). 